In [0]:
from pyspark.sql.types import *

schema = StructType([StructField('timestamp', TimestampType(), True), 
                     StructField('turbine_id', IntegerType(), True),
                     StructField('rotor_speed', DoubleType(), True),
                     StructField('blade_vibration', DoubleType(), True),
                     StructField('gearbox_temp', DoubleType(), True)])


aero_space_df = spark.readStream.format('csv').schema(schema).option('maxFilesPerRead', 1).load('/Volumes/workspace/training/training_volume/')

# display(aero_space_df, checkpointLocation = '/Volumes/workspace/training/training_volume')

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

aggregated_data = (
    aero_space_df.withWatermark("timestamp", "5 minutes")
    .groupBy(
        window(col("timestamp"), "5 minutes", "1 minute"), 'turbine_id')
    .agg(
        avg("rotor_speed").alias("rotor_speed_avg"),
        avg("blade_vibration").alias("blade_vibrations_avg"),
        avg("gearbox_temp").alias("gearbox_temp_avg"),
    )
)

In [0]:
display(aggregated_data, checkpointLocation = '/Volumes/workspace/training/training_volume')